In [2059]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import bellman_ford
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import re
import networkx as nx
from openpyxl import load_workbook
import random


In [4357]:
data = pd.read_excel("data/9.18.21.xlsx")
date = str(data["Date"][0].month) + "/" + str(data["Date"][0].day) + "/" + str(data["Date"][0].year)

tokens = []
for elt in data['Pool Tokens']:
    res = re.split(' / ', elt)
    if res[0] not in tokens:
        tokens.append(res[0])
    if res[1] not in tokens:
        tokens.append(res[1])
        
l = len(tokens)

In [4358]:
def construct_graph(data):
    G = nx.DiGraph()
    for elt in tokens:
        G.add_node(elt)
    for index, row in data[['Pool Tokens', 'Volume Token 0 (Excel)', 'Volume Token 1 (Excel)', 'Volume USD (Excel)', 'Fees USD (Excel)']].iterrows():
        token1, token2 = re.split(' / ', row[0])
        volume1 = row[1]
        volume2 = row[2]
#         if volume1 == volume2 and volume1 !=0 and volume2 !=0:
#             raise Exception("EQUALITY")
        if volume1 != 0 and volume2 !=0:
            if row[3] !=0:
                feetier = row[4]/row[3]
            else:
                feetier = 0.003
            #print(feetier)
            G.add_weighted_edges_from( [(token1, token2, -np.log(volume2/volume1 * (1-feetier)))] )
            G.add_weighted_edges_from( [(token2, token1, -np.log(volume1/volume2 * (1-feetier)))] )
    return G

In [4359]:
G = construct_graph(data)
def find_cycle(G, tokens):
    random.shuffle(tokens)
    for t in tokens:
        try:
            nx.find_negative_cycle(G, t)
        except Exception:
            pass
        else:
            cycle = nx.find_negative_cycle(G, t)
            if len(cycle)>3:
                return nx.find_negative_cycle(G, t)

In [4360]:
# WRITE PROGRAM THAT NUMERICALLY CALCULATES HOW MUCH YOU GAINED IN ARBITRAGE

In [4361]:
def uniswap_rate(A,B,alpha):
    return B/(A+alpha)

def revenue(q, cycle, dataset):
    pools = []
    for i in range(len(cycle)-1):
        token1 = cycle[i]
        token2 = cycle[i+1]
        if dataset.isin([token1 + " / " + token2]).any().any():
            pool =  dataset.loc[(dataset["Pool Tokens"] == token1 + " / " + token2)].iloc[-1]
            q1 = pool["Volume Token 0 (Excel)"]
            q2 = pool["Volume Token 1 (Excel)"]
        else:
            pool =  dataset.loc[(dataset["Pool Tokens"] == token2 + " / " + token1)].iloc[-1]
            q2 = pool["Volume Token 0 (Excel)"]
            q1 = pool["Volume Token 1 (Excel)"]
        
        volume = pool['Volume USD (Excel)']
        fees = pool['Fees USD (Excel)']
        
        if volume != 0:
            feetier = fees/volume
        else:
            feetier = 0.003
        
        
        pools.append([q1,q2,feetier])
    alpha = q
    for [q1,q2,feetier] in pools:
        if alpha > q1:
            return 0
        beta = (1-feetier) * alpha * uniswap_rate(q1,q2,alpha)
        alpha = beta
    return -beta

In [4362]:
def prof(q, cycle):
    return revenue(q, cycle, data)+q

while True:
    cycle = find_cycle(G, tokens)
    print(cycle)
    fun = minimize(prof, [0.1], cycle  ,method = 'Nelder-Mead', options={'maxiter':250, 'maxfev':500})
    if fun.success:
        print(fun)
        m = -fun.fun
        break;

['WBTC', 'XYO', 'UNI', 'WBTC']
['WBTC', 'XYO', 'UNI', 'WBTC']
['WBTC', 'MM', 'USDT', 'WBTC']
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -19.200403288925006
             x: [-3.791e-02]
           nit: 45
          nfev: 93
 final_simplex: (array([[-3.791e-02],
                       [-3.791e-02]]), array([-1.920e+01, -1.920e+01]))


In [4363]:
filename = "Profits.xlsx"
workbook = load_workbook(filename)
worksheet = workbook.active
data = {'Quantity': [m], 'Security': [cycle[0]], 'Date': [date]}
for record in list(zip(*data.values())):
    worksheet.append(record)
    
workbook.save(filename)

In [4364]:
m

19.200403288925006

In [3152]:
# TRYING TO ARBITRAGE EVERYTHING IN UNISWAP MARKET

In [3214]:
def trade(cycle, dataset):
    fun = minimize(prof, [0.0000001],cycle, method = 'Nelder-Mead')
    x = fun.x
    profit = -fun.fun
    if profit >0:
        for i in range(len(cycle)-1):
            token1 = cycle[i]
            token2 = cycle[i+1]
            if dataset.isin([token1 + " / " + token2]).any().any():
                pool =  dataset.loc[(dataset["Pool Tokens"] == token1 + " / " + token2)].iloc[-1]
                q1 = pool["Volume Token 0 (Excel)"]
                q2 = pool["Volume Token 1 (Excel)"]
                volume = pool['Volume USD (Excel)']
                fees = pool['Fees USD (Excel)']
                if volume != 0:
                    feetier = fees/volume
                else:
                    feetier = 0.003

                index = dataset.loc[(dataset["Pool Tokens"] == token1 + " / " + token2)].iloc[-1].name
                #print(dataset.at[index, "Volume Token 0 (Excel)"])
                dataset.at[index, "Volume Token 0 (Excel)"] = q1 + x
                dataset.at[index, "Volume Token 1 (Excel)"] = q2 - (1-feetier) * x * uniswap_rate(q1,q2,x)
                x = (1-feetier) * x * uniswap_rate(q1,q2,x)
            else:
                pool =  dataset.loc[(dataset["Pool Tokens"] == token2 + " / " + token1)].iloc[-1]
                q1 = pool["Volume Token 0 (Excel)"]
                q2 = pool["Volume Token 1 (Excel)"]
                volume = pool['Volume USD (Excel)']
                fees = pool['Fees USD (Excel)']
                if volume != 0:
                    feetier = fees/volume
                else:
                    feetier = 0.003

                index = dataset.loc[(dataset["Pool Tokens"] == token2 + " / " + token1)].iloc[-1].name
                #print(dataset.at[index, "Volume Token 0 (Excel)"])
                dataset.at[index, "Volume Token 0 (Excel)"] = q1 - (1-feetier) * x * uniswap_rate(q2,q1,x)
                dataset.at[index, "Volume Token 1 (Excel)"] = q2 + x
                x = (1-feetier) * x * uniswap_rate(q2,q1,x)
    else:
        profit = 0
    return dataset, profit

In [3215]:
profit=0
for _ in range(10):
    cycle = find_cycle(G, tokens)
    if len(cycle) <=3:
        break;
    data, p = trade(cycle, data)
    profit += p
    G = construct_graph(data)
    print(profit)

0.24016757037459513
33.6980788760594
46.07377639689297
1337913.3669153964
1337917.2522066978
1337917.252250512
1337917.2523281956
1337917.2523621034
1337970.1459281687
1337970.146943484
